# **A Revision of Some Fundamentals**

In [1]:
import torch
import matplotlib.pyplot as plt
import random

## Callbacks

### Callbacks as GUI Events

In [2]:
import ipywidgets as widgets

In [3]:
w = widgets.Button(description='Click here')
output = widgets.Output()

In [4]:
w

Button(description='Click here', style=ButtonStyle())

In [5]:
# Adding clickable call back
def f(o): 
    with output:
        print("Yep, that's a click.")

w.on_click(f)

In [6]:
display(w, output)

Button(description='Click here', style=ButtonStyle())

Output(outputs=({'name': 'stdout', 'text': "Yep, that's a click.\n", 'output_type': 'stream'},))

## Creating Custom Callbacks

In [7]:
from time import sleep

In [10]:
def slow_calculation():
    res = 0
    for i in range(5):
        res += i*i
        print(res)
        sleep(2)
    return res

In [11]:
%time slow_calculation()

0
1
5
14
30
CPU times: user 18 ms, sys: 2.4 ms, total: 20.4 ms
Wall time: 10 s


30

In [12]:
# Adding a callback with a print out after each cycle
def slow_calculation(cb=None):
    res = 0
    for i in range(5):
        res += i*i
        print(res)
        sleep(2)
        if cb: cb(i)
    return res

In [13]:
# Callback function
def show_progress(epoch): print(f"Epoch {epoch} completed!")

In [15]:
%time slow_calculation(show_progress)

0
Epoch 0 completed!
1
Epoch 1 completed!
5
Epoch 2 completed!
14
Epoch 3 completed!
30
Epoch 4 completed!
CPU times: user 22.8 ms, sys: 0 ns, total: 22.8 ms
Wall time: 10 s


30

## Lambdas and Partials

In [16]:
# Adding a lambda function for inplace function calls
slow_calculation(lambda o: print(f"Epoch {o} completed!"))

0
Epoch 0 completed!
1
Epoch 1 completed!
5
Epoch 2 completed!
14
Epoch 3 completed!
30
Epoch 4 completed!


30

In [17]:
def show_progress(exclamation, epoch): print(f"{exclamation}! Epoch {epoch} completed!")

In [18]:
slow_calculation(lambda o: show_progress("Slowly!", o))

0
Slowly!! Epoch 0 completed!
1
Slowly!! Epoch 1 completed!
5
Slowly!! Epoch 2 completed!
14
Slowly!! Epoch 3 completed!
30
Slowly!! Epoch 4 completed!


30

In [22]:
# For similar results, we can use an inner function to achieve similar results
def make_show_progress(exclamation):
    def _inner(epoch): print(f"{exclamation}! Epoch {epoch} completed!")
    return _inner

In [23]:
slow_calculation(make_show_progress("Inner function check!"))

0
Inner function check!! Epoch 0 completed!
1
Inner function check!! Epoch 1 completed!
5
Inner function check!! Epoch 2 completed!
14
Inner function check!! Epoch 3 completed!
30
Inner function check!! Epoch 4 completed!


30

In [24]:
# Moving on to partials
from functools import partial

In [26]:
slow_calculation(partial(show_progress, "Testing Testing"))

0
Testing Testing! Epoch 0 completed!
1
Testing Testing! Epoch 1 completed!
5
Testing Testing! Epoch 2 completed!
14
Testing Testing! Epoch 3 completed!
30
Testing Testing! Epoch 4 completed!


30

In [28]:
f2 = partial(show_progress, "Another partial")
f2

functools.partial(<function show_progress at 0x7fce2f775090>, 'Another partial')

## Callbacks as Callable Classes

In [56]:
class ProgressShowingCallback():
    def __init__(self, exclamation="Another test"): self.exclamation = exclamation
    def __call__(self, epoch): print(f"{self.exclamation}! Epoch {epoch} completed!")

In [57]:
cb = ProgressShowingCallback()

In [58]:
slow_calculation(cb)

0
Another test! Epoch 0 completed!
1
Another test! Epoch 1 completed!
5
Another test! Epoch 2 completed!
14
Another test! Epoch 3 completed!
30
Another test! Epoch 4 completed!


30

In [60]:
cb = ProgressShowingCallback("Override!")
slow_calculation(cb)

0
Override!! Epoch 0 completed!
1
Override!! Epoch 1 completed!
5
Override!! Epoch 2 completed!
14
Override!! Epoch 3 completed!
30
Override!! Epoch 4 completed!


30

## Multiple Callback Functions `*args` and `**kwargs`

In [67]:
# Function which returns positional and keyword args
# We don't necessarily have to stick to the same arg and kwarg names, they can really be anything
# as long as we are mindful of the usage of * and **
def f(*args, **kwargs): print(f"arguments(args): {args};\nkeyword argments(kwargs): {kwargs}")

In [68]:
f(33, 'bilal', somearg="chaudhry")

arguments(args): (33, 'bilal');
keyword argments(kwargs): {'somearg': 'chaudhry'}


In [69]:
# Create a function with a combo of args and kwargs
def g(a, b, c=22): print(a, b, c)

In [71]:
args = [1, 2]
kwargs = {'c': 431}

g(*args, **kwargs)

1 2 431


In [88]:
# Modifying the slow_calculation() function with two additional methods to extend the
# functionality of callback
def slow_calculation(cb=None):
    res = 0
    for i in range(5):
        if cb: cb.before_calc(i)
        res += i*i
        print(f"Result: {res}")
        sleep(2)
        if cb: cb.after_calc(i, val=res)
    return res

In [89]:
# Creating a class to handle the before and after callback calculation
class PrintStepCallback():
    def before_calc(self, *args, **kwargs): print(f"Warming Up!")
    def after_calc (self, *args, **kwargs): print(f"Step Completed!\n")

In [90]:
slow_calculation(PrintStepCallback())

Warming Up!
Result: 0
Step Completed!

Warming Up!
Result: 1
Step Completed!

Warming Up!
Result: 5
Step Completed!

Warming Up!
Result: 14
Step Completed!

Warming Up!
Result: 30
Step Completed!



30

In [93]:
# Let's also utilize the argument in the after calc function
class PrintStepCallback():
    def __init__(self): pass
    def before_calc(self, epoch, **kwargs): print(f"Warming Up!: {epoch}")
    def after_calc(self, epoch, val, **kwargs): print(f"Epoch {epoch} completed!.\nRepeating output {val}\n")

In [94]:
slow_calculation(PrintStepCallback())

Warming Up!: 0
Result: 0
Epoch 0 completed!.
Repeating output 0

Warming Up!: 1
Result: 1
Epoch 1 completed!.
Repeating output 1

Warming Up!: 2
Result: 5
Epoch 2 completed!.
Repeating output 5

Warming Up!: 3
Result: 14
Epoch 3 completed!.
Repeating output 14

Warming Up!: 4
Result: 30
Epoch 4 completed!.
Repeating output 30



30

## Modifying Behaviour

In [100]:
def slow_calculation(cb=None):
    res = 0
    for i in range(5):
        if cb and hasattr(cb, 'before_calc'): cb.before_calc(i)
        res += i*i 
        print(res)
        sleep(2)
        if cb and hasattr(cb, 'after_calc'): 
            if cb.after_calc(i, res):
                print("Emergency stop!!")
                break
    return

In [101]:
# Modifying the PrintAfterCallback class
class PrintAfterCallback():
    def after_calc(self, epoch, val):
        print(f"Epoch {epoch} completed! Repeated output {val}")
        if val > 10: return True

In [102]:
slow_calculation(PrintAfterCallback())

0
Epoch 0 completed! Repeated output 0
1
Epoch 1 completed! Repeated output 1
5
Epoch 2 completed! Repeated output 5
14
Epoch 3 completed! Repeated output 14
Emergency stop!!


In [121]:
# Why not chuck it all into a class
class SlowCalculator():
    def __init__(self, cb=None): self.cb, self.res = cb, 0

    def callback(self, cb_name, *args):
        if not self.cb: return
        cb = getattr(self.cb, cb_name, None)
        if cb: return cb(self, *args)

    def calc(self):
        for i in range(5):
            self.callback('before_calc', i)
            self.res += i*i
            sleep(2)
            if self.callback('after_calc', i):
                print("Emergency Stop!")
                break

In [127]:
# Modified callbacks
class ModifyingCallback():
    def after_calc(self, calc, epoch):
        print(f"Epoch {epoch} completed!\nModified Output {calc.res}\n")
        if calc.res > 10: return True
        if calc.res < 3: calc.res = calc.res*2

In [128]:
calculator = SlowCalculator(ModifyingCallback())

In [129]:
calculator.calc()

Epoch 0 completed!
Modified Output 0

Epoch 1 completed!
Modified Output 1

Epoch 2 completed!
Modified Output 6

Epoch 3 completed!
Modified Output 15

Emergency Stop!


## `__dunder__` Methods

In [130]:
class SloppyAdder():
    def __init__(self, o): self.o = o
    def __add__(self, b): return SloppyAdder(self.o + b.o + 0.0121)
    def __repr__(self): return str(f"Output: {self.o}")

In [131]:
a = SloppyAdder(22)
b = SloppyAdder(3)

a+b

Output: 25.0121

In [133]:
class A: a, b  = 3, 4

In [134]:
a = A()

In [140]:
# Fine grained and dynamic control
getattr(a, 'b' if random.random() > 0.5 else 'a')

4

In [141]:
class B:
    a, b = 1, 5
    def __getattr__(self, k):
        if k[0] == '_': raise AttributeError(k)
        return f'Test case {k}'

In [142]:
b = B()

In [144]:
b.a, b.b

(1, 5)

In [145]:
b.blah

'Test case blah'